<a href="https://colab.research.google.com/github/SandraMangas/AnalliticaDatos_ProyectoSucre/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Importar Librerias

In [218]:
import pandas as pd  # Para manipulación y análisis de datos
import numpy as np # Para operaciones numéricas
import matplotlib.pyplot as plt # Para crear visualizaciones estáticas
import seaborn as sns # Para crear gráficos estadísticos
import requests # Trae datos de un sitio web o API

## 2. Cargar Datos

### 2.1 Intituciones Educativas

In [219]:
# URL de la API OData
url_institucionesE = "https://www.datos.gov.co/api/odata/v4/j7te-zasu"

# Solicitud a la API
response = requests.get(url_institucionesE) # requests.get() obtiene los datos JSON desde la API.obtiene los datos JSON desde la API.
data_url_institucionesE = response.json()

# Convertir los datos de 'value' a DataFrame legible
df_institucionesE = pd.json_normalize(data_url_institucionesE['value']) # data['value'] accede a la lista de registros reales.

### 2.2 Conectividad

In [220]:
# URL de la API OData
url_conectividad = "https://www.datos.gov.co/api/odata/v4/a2f2-cnws"

# Solicitud a la API
response = requests.get(url_conectividad) # requests.get() obtiene los datos JSON desde la API.obtiene los datos JSON desde la API.
data_url_conectividad = response.json()

# Convertir los datos de 'value' a DataFrame
df_conectividad = pd.json_normalize(data_url_conectividad['value']) # data['value'] accede a la lista de registros reales.

### 2.3 Pruebas Saber 11

#### 2.3.1 Dataset de Datos Abiertos

In [221]:
# URL la API con extensión JSON y límite
url_saber11 = "https://www.datos.gov.co/resource/kgxf-xxbe.json?$limit=50000"

# Solicitud a la API
response = requests.get(url_saber11)
data_url_saber11 = response.json()

# Convertir directamente a DataFrame
df_saber11 = pd.DataFrame(data_url_saber11)

# Nota importante: Si se usa .json, no se requiere .odata ni .api.
# https://www.datos.gov.co/api/odata/v4/kgxf-xxbe

In [222]:
print(df_saber11.columns)

Index(['periodo', 'estu_tipodocumento', 'estu_consecutivo',
       'cole_area_ubicacion', 'cole_bilingue', 'cole_calendario',
       'cole_caracter', 'cole_cod_dane_establecimiento', 'cole_cod_dane_sede',
       'cole_cod_depto_ubicacion', 'cole_cod_mcpio_ubicacion',
       'cole_codigo_icfes', 'cole_depto_ubicacion', 'cole_genero',
       'cole_jornada', 'cole_mcpio_ubicacion', 'cole_naturaleza',
       'cole_nombre_establecimiento', 'cole_nombre_sede',
       'cole_sede_principal', 'estu_cod_depto_presentacion',
       'estu_cod_mcpio_presentacion', 'estu_cod_reside_depto',
       'estu_cod_reside_mcpio', 'estu_depto_presentacion', 'estu_depto_reside',
       'estu_estadoinvestigacion', 'estu_estudiante', 'estu_fechanacimiento',
       'estu_genero', 'estu_mcpio_presentacion', 'estu_mcpio_reside',
       'estu_nacionalidad', 'estu_pais_reside', 'estu_privado_libertad',
       'fami_cuartoshogar', 'fami_educacionmadre', 'fami_educacionpadre',
       'fami_estratovivienda', 'fami_perso

In [223]:
df_saber11["cole_depto_ubicacion"] = df_saber11["cole_depto_ubicacion"].str.strip().str.upper()
# Filtrar solo las filas donde el departamento sea SUCRE
# Asegúrate que el nombre esté escrito exactamente así:
df_saber11_sucre = df_saber11[df_saber11["cole_depto_ubicacion"] == "SUCRE"]

#### 2.3.2 Dataset de Repositorio ICFES

In [224]:
# Cargar el dataset 'PruebasSaber11.csv' desde GitHub.
# Este archivo fue transformado previamente con Power Query: se eliminaron columnas irrelevantes
# y se filtraron los datos del departamento de Sucre.

url_saber11_icfes = "https://raw.githubusercontent.com/SandraMangas/AnalliticaDatos_ProyectoSucre/main/PruebasSaber11.csv"
df_saber11_icfes = pd.read_csv(url_saber11_icfes, encoding='latin1', sep=';')

## 3. Exploración Inicial

### 3.1 Intituciones Educativas

In [225]:
df_institucionesE.head(20)

,__id,nombre_establecimiento,codigo_dane_sede,nombre_sede,zona
0,row-ey8z~kj7g~u8nv,INST EDUC BUENAVISTA,170110000025,INST EDUC BUENAVISTA,Urbano
1,row-ejkd.f47a~qiua,INST EDUC BUENAVISTA,170110000033,SEDE NUESTRA SEÑORA DEL ROSARIO,Urbano
2,row-inth.2vhr_qrty,INST EDUC BUENAVISTA,170110000041,SEDE SAN FRANCISCO DE ASIS,Urbano
3,row-f5bn.cir7~hizu,INST EDUC BUENAVISTA,170110000050,SEDE MARIA AUXILIADORA,Urbano
4,row-nxx4_bjci-bsqm,INST EDUC SAN JUAN BAUTISTA,170124000001,SEDE ROSA GARCIA,Urbano
5,row-nr9f_qnjr-wzdk,INST EDUC SAN JUAN BAUTISTA,170124000010,SEDE SAN VICENTE FERRER,Urbano
6,row-85zw~z36b_4csg,INST EDUC SAN JUAN BAUTISTA,170124000192,SEDE SANTA TERESITA DEL NIÑO JESUS,Urbano
7,row-k25g-yhmr.4ig3,INST EDUC SAN JUAN BAUTISTA,170124000265,INST EDUC SAN JUAN BAUTISTA,Urbano
8,row-mjvn_sigp-hfb3,I.E. VICTOR ZUBIRIA,170204000053,I.E. VICTOR ZUBIRIA - SEDE PRINCIPAL,Urbano
9,row-876m.pia4~paax,I.E. VICTOR ZUBIRIA,170204000061,SEDE URBANA DE NIÑAS,Urbano


In [226]:
df_institucionesE.tail(20)

,__id,nombre_establecimiento,codigo_dane_sede,nombre_sede,zona
780,row-hjgm-336m-ht5p,I.E. SANTA TERESITA,370820000112,I.E. SANTA TERESITA - SEDE PRINCIPAL,Urbano
781,row-sgc2.ibvg_uwqk,I.E. EL MAMON,270215000297,CANTAGALLO,Rural
782,row-uvnc~2cic.k8zg,I.E. EL MAMON,270215000866,LAS TINAS,Rural
783,row-kfhc.q2u9~4kfq,I.E. EL MAMON,270215001293,RINCON DE LAS FLORES,Rural
784,row-7sbp_gbe4~tzhi,I.E. EL MAMON,470215001110,I.E. EL MAMON - SEDE PRINCIPAL,Rural
785,row-tu6f_3mng-34p4,CENTRO EDUCATIVO LOS COROZOS,270429000847,CENTRO EDUCATIVO SANTISIMA CRUZ DE HUIRA ARRIBA,Rural
786,row-5f2s-9nv3-fwmd,CENTRO EDUCATIVO LOS COROZOS,270429001533,CENTRO EDUCATIVO LA MILAGROSA,Rural
787,row-46ha.spmb.hh38,CENTRO EDUCATIVO LOS COROZOS,270429002220,CENTRO EDUCATIVO SAN JOSE HUIRA ABAJO,Rural
788,row-mfvp~5vnq-z45g,CENTRO EDUCATIVO LOS COROZOS,470429001966,CENTRO EDUCATIVO LOS COROZOS,Rural
789,row-ejjj~gawc~mjqy,I.E. PALO ALTO,170713000879,PITA CAPACHO NO. 2,Rural


In [227]:
df_institucionesE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   __id                    800 non-null    object
 1   nombre_establecimiento  800 non-null    object
 2   codigo_dane_sede        800 non-null    int64 
 3   nombre_sede             800 non-null    object
 4   zona                    800 non-null    object
dtypes: int64(1), object(4)
memory usage: 31.4+ KB


### 3.2 Conectividad

In [228]:
df_conectividad.head()

,__id,nombre_departamento,nombre_municipio,pdet,nombre_centro_poblado,nombre_sede_educativa,fecha_inicio_de_operaci_n,fin_de_operaci_n_inicial,estado
0,row-x3fv_7fq4.tx3x,SUCRE,MAJAGUAL,NO,CORREG EL NARANJO,INST EDUC EL NARANJO,None,None,INSTALACIÓN
1,row-5rbh.85xj_2mc9,SUCRE,OVEJAS,SI,VEREDA DAMASCO,CENTRO EDUCATIVO DAMASCO,2021-09-25T00:00:00.000,2031-09-25T00:00:00.000,OPERACIÓN
2,row-vpvn_jhri.m68b,SUCRE,OVEJAS,SI,VEREDA VILLA COLOMBIA,CENTRO EDUCATIVO VILLA COLOMBIA,2021-09-25T00:00:00.000,2031-09-25T00:00:00.000,OPERACIÓN
3,row-65em.fubm-dvu4,SUCRE,TOLÚ VIEJO,SI,CORREG DE LAS PIEDRAS,I.E. TÉCNICO AGROPECUARIO DE LAS PIEDRAS - SED...,None,None,INSTALACIÓN
4,row-er2d.32h4.7fje,SUCRE,TOLÚ VIEJO,SI,CORREG LA PALMIRA,INSTITUCION EDUCATIVA PALMIRA,None,None,INSTALACIÓN


In [229]:
df_conectividad.tail()

,__id,nombre_departamento,nombre_municipio,pdet,nombre_centro_poblado,nombre_sede_educativa,fecha_inicio_de_operaci_n,fin_de_operaci_n_inicial,estado
470,row-639p-mi9q_cjp7,SUCRE,GUARANDA,NO,VDA. LAS MERCEDES,LAS MERCEDES,None,None,EN PLANEACIÓN
471,row-ium6~g7ec.8qki,SUCRE,GUARANDA,NO,VDA. PUEBLO NUEVO,PUEBLO NUEVO,None,None,INSTALACIÓN
472,row-hsbd_3sav~txc2,SUCRE,CHALÁN,SI,DESBARRANCADO,LOS COMUNEROS,None,None,EN PLANEACIÓN
473,row-bu3d.kbb2_jegg,SUCRE,COROZAL,NO,VEREDA LA LOMA,LA LOMA,None,None,INSTALACIÓN
474,row-ffu2.rtx6~vzda,SUCRE,COLOSÓ,SI,CORREG CEIBA,CEIBA,2022-01-01T00:00:00.000,2032-01-01T00:00:00.000,OPERACIÓN


In [230]:
df_conectividad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   __id                       475 non-null    object
 1   nombre_departamento        475 non-null    object
 2   nombre_municipio           475 non-null    object
 3   pdet                       475 non-null    object
 4   nombre_centro_poblado      475 non-null    object
 5   nombre_sede_educativa      475 non-null    object
 6   fecha_inicio_de_operaci_n  179 non-null    object
 7   fin_de_operaci_n_inicial   179 non-null    object
 8   estado                     475 non-null    object
dtypes: object(9)
memory usage: 33.5+ KB


### 3.3 Saber 11

#### 3.3.1 Dataset de Datos Abiertos

In [231]:
df_saber11.head()

,periodo,estu_tipodocumento,estu_consecutivo,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_depto_ubicacion,...,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
0,20131,CR,SB11201310000414,URBANO,N,B,ACADÉMICO,311848000812,311848000812,11,...,Si,Si,Si,B+,94,88,NaN,NaN,NaN,NaN
1,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,141016000305,241016000342,41,...,Si,Si,Si,B1,71,66,70,65,69,339
2,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,141016000305,241016000342,41,...,Si,Si,Si,B1,71,66,70,65,69,339
3,20122,TI,SB11201220204399,URBANO,N,A,TÉCNICO/ACADÉMICO,163130000718,163130000718,63,...,Si,Si,No,A1,48,45,NaN,NaN,NaN,NaN
4,20132,TI,SB11201320464198,URBANO,N,A,TÉCNICO,119001000559,119001000559,19,...,Si,Si,Si,A-,43,52,NaN,NaN,NaN,NaN


In [232]:
df_saber11.tail()

,periodo,estu_tipodocumento,estu_consecutivo,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_depto_ubicacion,...,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
49995,20194,TI,SB11201940623464,RURAL,NaN,A,NaN,418001800021,418001800021,18,...,Si,Si,Si,A-,47,45,45,45,54,236
49996,20132,TI,SB11201320493041,URBANO,NaN,A,ACADÉMICO,373411002242,373411002242,73,...,Si,No,Si,A1,51,49,NaN,NaN,NaN,NaN
49997,20112,TI,SB11201120318868,URBANO,N,A,ACADÉMICO,311001011800,311001011800,11,...,Si,Si,Si,A1,49,52,NaN,NaN,NaN,NaN
49998,20131,TI,SB11201310079267,URBANO,N,A,TÉCNICO/ACADÉMICO,386568061015,386568061015,86,...,No,No,No,A-,27,44,NaN,NaN,NaN,NaN
49999,20132,TI,SB11201320070946,URBANO,N,A,ACADÉMICO,115001000057,115001000057,15,...,No,No,No,A2,59,74,NaN,NaN,NaN,NaN


In [233]:
df_saber11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   periodo                        50000 non-null  object
 1   estu_tipodocumento             50000 non-null  object
 2   estu_consecutivo               50000 non-null  object
 3   cole_area_ubicacion            49977 non-null  object
 4   cole_bilingue                  44133 non-null  object
 5   cole_calendario                49987 non-null  object
 6   cole_caracter                  49460 non-null  object
 7   cole_cod_dane_establecimiento  49977 non-null  object
 8   cole_cod_dane_sede             49997 non-null  object
 9   cole_cod_depto_ubicacion       49998 non-null  object
 10  cole_cod_mcpio_ubicacion       49998 non-null  object
 11  cole_codigo_icfes              49809 non-null  object
 12  cole_depto_ubicacion           49998 non-null  object
 13  c

In [234]:
df_saber11_sucre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 21 to 49925
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   periodo                        907 non-null    object
 1   estu_tipodocumento             907 non-null    object
 2   estu_consecutivo               907 non-null    object
 3   cole_area_ubicacion            907 non-null    object
 4   cole_bilingue                  778 non-null    object
 5   cole_calendario                907 non-null    object
 6   cole_caracter                  906 non-null    object
 7   cole_cod_dane_establecimiento  907 non-null    object
 8   cole_cod_dane_sede             907 non-null    object
 9   cole_cod_depto_ubicacion       907 non-null    object
 10  cole_cod_mcpio_ubicacion       907 non-null    object
 11  cole_codigo_icfes              905 non-null    object
 12  cole_depto_ubicacion           907 non-null    object
 13  cole_ge

### 3.3.2 Dataset de Repositorio ICFES

In [235]:
df_saber11_icfes.head()

,periodo,cole_area_ubicacion,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_depto_ubicacion,cole_jornada,cole_mcpio_ubicacion,cole_naturaleza,cole_nombre_establecimiento,cole_nombre_sede,...,estu_depto_presentacion,estu_depto_reside,estu_mcpio_presentacion,estu_mcpio_reside,punt_c_naturales,punt_global,punt_ingles,punt_lectura_critica,punt_matematicas,punt_sociales_ciudadanas
0,20241,URBANO,370670800018,370670800018,SUCRE,SABATINA,SAMPUÉS,NO OFICIAL,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,...,SUCRE,SUCRE,SINCELEJO,SAMPUÉS,39,202,39.0,43,36,44
1,20241,URBANO,370670800018,370670800018,SUCRE,SABATINA,SAMPUÉS,NO OFICIAL,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,...,SUCRE,SUCRE,SINCELEJO,SAMPUÉS,42,232,52.0,56,49,37
2,20241,URBANO,370670800018,370670800018,SUCRE,SABATINA,SAMPUÉS,NO OFICIAL,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,...,SUCRE,SUCRE,SINCELEJO,SAMPUÉS,44,206,32.0,47,36,41
3,20241,URBANO,370670800018,370670800018,SUCRE,SABATINA,SAMPUÉS,NO OFICIAL,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,...,SUCRE,SUCRE,SINCELEJO,SAMPUÉS,33,177,31.0,37,41,32
4,20241,URBANO,370670800018,370670800018,SUCRE,SABATINA,SAMPUÉS,NO OFICIAL,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,INSTITUCION EDUCATIVA GIMNASIO PEDAGOGICO SAMP...,...,SUCRE,SUCRE,SINCELEJO,SAMPUÉS,35,186,45.0,42,41,28


In [236]:
df_saber11_icfes.tail()

,periodo,cole_area_ubicacion,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_depto_ubicacion,cole_jornada,cole_mcpio_ubicacion,cole_naturaleza,cole_nombre_establecimiento,cole_nombre_sede,...,estu_depto_presentacion,estu_depto_reside,estu_mcpio_presentacion,estu_mcpio_reside,punt_c_naturales,punt_global,punt_ingles,punt_lectura_critica,punt_matematicas,punt_sociales_ciudadanas
11774,20242,RURAL,270678000580,270678000580,SUCRE,MAÑANA,GALERAS,OFICIAL,INST EDUC PUERTO FRANCO,INST EDUC PUERTO FRANCO,...,SUCRE,SUCRE,GALERAS,GALERAS,45,206,26.0,41,39,45
11775,20242,RURAL,270702000120,270702000120,SUCRE,MAÑANA,SAN JUAN DE BETULIA,OFICIAL,I.E. SABANETA,I.E. SABANETA - SEDE PRINCIPAL,...,SUCRE,SUCRE,COROZAL,SAN JUAN DE BETULIA,49,229,52.0,48,39,45
11776,20242,URBANO,170717000016,170717000016,SUCRE,MAÑANA,SAN PEDRO,OFICIAL,INST EDUC SAN PEDRO CLAVER,INST EDUC SAN PEDRO CLAVER,...,SUCRE,SUCRE,SAN PEDRO,SAN PEDRO,49,229,52.0,54,45,33
11777,20242,URBANO,170215000977,170215000977,SUCRE,TARDE,COROZAL,OFICIAL,I.E. NORMAL SUPERIOR DE COROZAL,I.E. NORMAL SUPERIOR DE COROZAL - SEDE PRINCIPAL,...,SUCRE,SUCRE,COROZAL,COROZAL,58,279,54.0,59,56,51
11778,20242,URBANO,170204000053,170204000053,SUCRE,MAÑANA,COLOSÓ,OFICIAL,I.E. VICTOR ZUBIRIA,I.E. VICTOR ZUBIRIA - SEDE PRINCIPAL,...,SUCRE,SUCRE,SINCELEJO,COLOSÓ,61,328,57.0,63,68,73


In [237]:
df_saber11_icfes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11779 entries, 0 to 11778
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   periodo                        11779 non-null  int64  
 1   cole_area_ubicacion            11779 non-null  object 
 2   cole_cod_dane_establecimiento  11779 non-null  int64  
 3   cole_cod_dane_sede             11779 non-null  int64  
 4   cole_depto_ubicacion           11779 non-null  object 
 5   cole_jornada                   11779 non-null  object 
 6   cole_mcpio_ubicacion           11779 non-null  object 
 7   cole_naturaleza                11779 non-null  object 
 8   cole_nombre_establecimiento    11779 non-null  object 
 9   cole_nombre_sede               11779 non-null  object 
 10  desemp_c_naturales             11779 non-null  int64  
 11  desemp_ingles                  11696 non-null  object 
 12  desemp_lectura_critica         11779 non-null 

## 4. Calidad del Dataset

### 4.1 Intituciones Educativas

In [238]:
# Cantidad de datos nulos
df_institucionesE.isnull().sum()

,0
__id,0
nombre_establecimiento,0
codigo_dane_sede,0
nombre_sede,0
zona,0


In [239]:
# Porcentaje de datos nulos
print('Porcentaje datos nulos')
df_institucionesE.isnull().mean()*100

Porcentaje datos nulos


,0
__id,0.0
nombre_establecimiento,0.0
codigo_dane_sede,0.0
nombre_sede,0.0
zona,0.0


In [240]:
# Porcentaje de datos no nulos
print('Porcentaje datos no nulos')
(1-df_institucionesE.isnull().mean())*100

Porcentaje datos no nulos


,0
__id,100.0
nombre_establecimiento,100.0
codigo_dane_sede,100.0
nombre_sede,100.0
zona,100.0


In [241]:
total_datos_institucionesE=df_institucionesE.shape[0]*df_institucionesE.shape[1]
total_nulos_institucionesE=df_institucionesE.isnull().sum().sum()
print('Porcentaje datos nulo en el dataset',(total_nulos_institucionesE/total_datos_institucionesE)*100)

Porcentaje datos nulo en el dataset 0.0


In [242]:
print('Porcentaje completitud del dataset', (1-total_nulos_institucionesE/total_datos_institucionesE)*100)

Porcentaje completitud del dataset 100.0


### 4.2 Conectividad

In [243]:
# Cantidad de datos nulos
df_conectividad.isnull().sum()

,0
__id,0
nombre_departamento,0
nombre_municipio,0
pdet,0
nombre_centro_poblado,0
nombre_sede_educativa,0
fecha_inicio_de_operaci_n,296
fin_de_operaci_n_inicial,296
estado,0


In [244]:
# Porcentaje de datos nulos
print('Porcentaje datos nulos')
df_conectividad.isnull().mean()*100

Porcentaje datos nulos


,0
__id,0.000000
nombre_departamento,0.000000
nombre_municipio,0.000000
pdet,0.000000
nombre_centro_poblado,0.000000
nombre_sede_educativa,0.000000
fecha_inicio_de_operaci_n,62.315789
fin_de_operaci_n_inicial,62.315789
estado,0.000000


In [245]:
# Porcentaje de datos no nulos
print('Porcentaje datos no nulos')
(1-df_conectividad.isnull().mean())*100

Porcentaje datos no nulos


,0
__id,100.000000
nombre_departamento,100.000000
nombre_municipio,100.000000
pdet,100.000000
nombre_centro_poblado,100.000000
nombre_sede_educativa,100.000000
fecha_inicio_de_operaci_n,37.684211
fin_de_operaci_n_inicial,37.684211
estado,100.000000


In [246]:
total_datos_conectividad=df_conectividad.shape[0]*df_conectividad.shape[1]
total_nulos_conectividad=df_conectividad.isnull().sum().sum()
print('Porcentaje datos nulo en el dataset',(total_nulos_conectividad/total_datos_conectividad)*100)

Porcentaje datos nulo en el dataset 13.84795321637427


In [247]:
print('Porcentaje completitud del dataset', (1-total_nulos_conectividad/total_datos_conectividad)*100)

Porcentaje completitud del dataset 86.15204678362574


### 4.3 Pruebas Saber 11

#### 4.3.1 Dataset de Datos Abiertos

In [248]:
# Cantidad de datos nulos
df_saber11.isnull().sum()

,0
periodo,0
estu_tipodocumento,0
estu_consecutivo,0
cole_area_ubicacion,23
cole_bilingue,5867
cole_calendario,13
cole_caracter,540
cole_cod_dane_establecimiento,23
cole_cod_dane_sede,3
cole_cod_depto_ubicacion,2


In [249]:
df_saber11_sucre.isnull().sum()

,0
periodo,0
estu_tipodocumento,0
estu_consecutivo,0
cole_area_ubicacion,0
cole_bilingue,129
cole_calendario,0
cole_caracter,1
cole_cod_dane_establecimiento,0
cole_cod_dane_sede,0
cole_cod_depto_ubicacion,0


In [250]:
# Porcentaje de datos nulos
print('Porcentaje datos nulos')
df_saber11.isnull().mean()*100

Porcentaje datos nulos


,0
periodo,0.000
estu_tipodocumento,0.000
estu_consecutivo,0.000
cole_area_ubicacion,0.046
cole_bilingue,11.734
cole_calendario,0.026
cole_caracter,1.080
cole_cod_dane_establecimiento,0.046
cole_cod_dane_sede,0.006
cole_cod_depto_ubicacion,0.004


In [251]:
# Porcentaje de datos nulos
print('Porcentaje datos nulos')
df_saber11_sucre.isnull().mean()*100

Porcentaje datos nulos


,0
periodo,0.000000
estu_tipodocumento,0.000000
estu_consecutivo,0.000000
cole_area_ubicacion,0.000000
cole_bilingue,14.222712
cole_calendario,0.000000
cole_caracter,0.110254
cole_cod_dane_establecimiento,0.000000
cole_cod_dane_sede,0.000000
cole_cod_depto_ubicacion,0.000000


In [252]:
# Porcentaje de datos no nulos
print('Porcentaje datos no nulos')
(1-df_saber11.isnull().mean())*100

Porcentaje datos no nulos


,0
periodo,100.000
estu_tipodocumento,100.000
estu_consecutivo,100.000
cole_area_ubicacion,99.954
cole_bilingue,88.266
cole_calendario,99.974
cole_caracter,98.920
cole_cod_dane_establecimiento,99.954
cole_cod_dane_sede,99.994
cole_cod_depto_ubicacion,99.996


In [253]:
# Porcentaje de datos no nulos
print('Porcentaje datos no nulos')
(1-df_saber11_sucre.isnull().mean())*100

Porcentaje datos no nulos


,0
periodo,100.000000
estu_tipodocumento,100.000000
estu_consecutivo,100.000000
cole_area_ubicacion,100.000000
cole_bilingue,85.777288
cole_calendario,100.000000
cole_caracter,99.889746
cole_cod_dane_establecimiento,100.000000
cole_cod_dane_sede,100.000000
cole_cod_depto_ubicacion,100.000000


In [254]:
total_datos_saber11=df_saber11.shape[0]*df_saber11.shape[1]
total_nulos_saber11=df_saber11.isnull().sum().sum()
print('Porcentaje datos nulo en el dataset',(total_nulos_saber11/total_datos_saber11)*100)

Porcentaje datos nulo en el dataset 4.016117647058824


In [255]:
total_datos_saber11_sucre=df_saber11_sucre.shape[0]*df_saber11_sucre.shape[1]
total_nulos_saber11_sucre=df_saber11_sucre.isnull().sum().sum()
print('Porcentaje datos nulo en el dataset',(total_nulos_saber11_sucre/total_datos_saber11_sucre)*100)

Porcentaje datos nulo en el dataset 3.97345266662343


In [256]:
print('Porcentaje completitud del dataset', (1-total_nulos_saber11/total_datos_saber11)*100)

Porcentaje completitud del dataset 95.98388235294118


In [257]:
print('Porcentaje completitud del dataset', (1-total_nulos_saber11_sucre/total_datos_saber11_sucre)*100)

Porcentaje completitud del dataset 96.02654733337657


#### 4.3.2 Dataset de Repositorio ICFES

In [258]:
# Cantidad de datos nulos
df_saber11_icfes.isnull().sum()

,0
periodo,0
cole_area_ubicacion,0
cole_cod_dane_establecimiento,0
cole_cod_dane_sede,0
cole_depto_ubicacion,0
cole_jornada,0
cole_mcpio_ubicacion,0
cole_naturaleza,0
cole_nombre_establecimiento,0
cole_nombre_sede,0


In [259]:
# Porcentaje de datos nulos
print('Porcentaje datos nulos')
df_saber11_icfes.isnull().mean()*100

Porcentaje datos nulos


,0
periodo,0.000000
cole_area_ubicacion,0.000000
cole_cod_dane_establecimiento,0.000000
cole_cod_dane_sede,0.000000
cole_depto_ubicacion,0.000000
cole_jornada,0.000000
cole_mcpio_ubicacion,0.000000
cole_naturaleza,0.000000
cole_nombre_establecimiento,0.000000
cole_nombre_sede,0.000000


In [260]:
# Porcentaje de datos no nulos
print('Porcentaje datos no nulos')
(1-df_saber11_icfes.isnull().mean())*100

Porcentaje datos no nulos


,0
periodo,100.000000
cole_area_ubicacion,100.000000
cole_cod_dane_establecimiento,100.000000
cole_cod_dane_sede,100.000000
cole_depto_ubicacion,100.000000
cole_jornada,100.000000
cole_mcpio_ubicacion,100.000000
cole_naturaleza,100.000000
cole_nombre_establecimiento,100.000000
cole_nombre_sede,100.000000


In [261]:
total_datos_saber11_icfes=df_saber11_icfes.shape[0]*df_saber11_icfes.shape[1]
total_nulos_df_saber11_icfes=df_saber11_icfes.isnull().sum().sum()
print('Porcentaje datos nulo en el dataset',(total_nulos_df_saber11_icfes/total_datos_saber11_icfes)*100)

Porcentaje datos nulo en el dataset 0.9609065725883792


In [262]:
print('Porcentaje completitud del dataset', (1-total_nulos_df_saber11_icfes/total_datos_saber11_icfes)*100)

Porcentaje completitud del dataset 99.03909342741161
